# Acceleration Example

### Imports

In [1]:
import org.apache.spark.streaming.{Seconds, Minutes, StreamingContext}
import org.apache.kafka.common.serialization.{BytesDeserializer, StringDeserializer}
import org.apache.spark.streaming.kafka010.KafkaUtils
import org.apache.spark.streaming.kafka010.LocationStrategies.PreferConsistent
import org.apache.spark.streaming.kafka010.ConsumerStrategies.Subscribe
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper
import org.apache.kafka.clients.producer.{KafkaProducer, ProducerConfig, ProducerRecord}
import collection.JavaConverters.mapAsJavaMapConverter

In [2]:
val ssc = new StreamingContext(sc, Seconds(1))
ssc.remember(Minutes(1))

In [3]:
val consumerParams = Map[String, Object](
  "bootstrap.servers" -> "kafka:9092",
  "key.deserializer" -> classOf[BytesDeserializer],
  "value.deserializer" -> classOf[StringDeserializer],
  "group.id" -> "spark-notebook",
  "auto.offset.reset" -> "earliest",
  "enable.auto.commit" -> (false: java.lang.Boolean)
)

val topics = Array("android")
val stream = KafkaUtils.createDirectStream[String, String](
  ssc,
  PreferConsistent,
  Subscribe[String, String](topics, consumerParams)
)

In [11]:
import scala.util.Try

stream.foreachRDD { rdd =>
  Try {
      spark.read.json(rdd.map(_.value())).createOrReplaceTempView("locations")
      spark.sql("select avg(x) as x, avg(y) as y, avg(z) as z, min(timestamp) as timestamp from locations").toJSON.foreachPartition {
          partition =>

          val props = new java.util.HashMap[String, Object]()
          props.put(ProducerConfig.BOOTSTRAP_SERVERS_CONFIG, "kafka:9092")
          props.put(ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG,
            "org.apache.kafka.common.serialization.StringSerializer")
          props.put(ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG,
            "org.apache.kafka.common.serialization.StringSerializer")

          val producer = new KafkaProducer[String, String](props)

          partition.foreach { s => 
              val message = new ProducerRecord[String, String]("acceleration", s)
              producer.send(message)
          }

          producer.close()
      }
    }
}

Name: java.lang.IllegalStateException
Message: Adding new inputs, transformations, and output operations after starting a context is not supported
StackTrace:   at org.apache.spark.streaming.dstream.DStream.validateAtInit(DStream.scala:223)
  at org.apache.spark.streaming.dstream.DStream.<init>(DStream.scala:65)
  at org.apache.spark.streaming.dstream.ForEachDStream.<init>(ForEachDStream.scala:39)
  at org.apache.spark.streaming.dstream.DStream.org$apache$spark$streaming$dstream$DStream$$foreachRDD(DStream.scala:653)
  at org.apache.spark.streaming.dstream.DStream$$anonfun$foreachRDD$1.apply$mcV$sp(DStream.scala:627)
  at org.apache.spark.streaming.dstream.DStream$$anonfun$foreachRDD$1.apply(DStream.scala:625)
  at org.apache.spark.streaming.dstream.DStream$$anonfun$foreachRDD$1.apply(DStream.scala:625)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.s

In [9]:
ssc.start()
ssc.awaitTerminationOrTimeout(20000)

Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`x`' given input columns: []; line 1 pos 95;
'Project ['avg('x) AS x#758, 'avg('y) AS y#759, 'avg('z) AS z#760, 'min('timestamp) AS timestamp#761]
+- 'Filter ('x > 5)
   +- SubqueryAlias locations
      +- LogicalRDD

StackTrace: 'Project ['avg('x) AS x#758, 'avg('y) AS y#759, 'avg('z) AS z#760, 'min('timestamp) AS timestamp#761]
+- 'Filter ('x > 5)
   +- SubqueryAlias locations
      +- LogicalRDD

  at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
  at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:77)
  at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:74)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:308)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfu

In [7]:
%%SQL
select * from locations

++
||
++
++

